In [1]:
import pandas


train_val = pandas.read_csv("../data/vn30/GAS_train.csv")
train_val

,time,open,high,low,close,volume
0,2019-01-02,54.33,54.83,54.33,54.77,417520
1,2019-01-03,54.58,54.89,53.00,53.44,404480
2,2019-01-04,52.87,53.95,51.99,53.32,336550
3,2019-01-07,54.58,55.08,53.95,54.45,255840
4,2019-01-08,54.45,55.97,54.45,55.71,600270
...,...,...,...,...,...,...
1245,2023-12-25,67.98,69.80,67.98,69.61,1057401
1246,2023-12-26,69.80,69.80,68.89,69.52,756305
1247,2023-12-27,69.52,69.80,69.25,69.43,707087
1248,2023-12-28,69.52,69.52,68.80,69.16,767974


In [2]:
train_val.drop("time", axis=1, inplace=True)

In [3]:
from typing import List, Tuple


CONTEXT_SIZE = 4


def extract_xy(table: pandas.DataFrame, *, context_size: int) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    series: List[pandas.Series] = []
    original = table.columns
    output_names: List[str] = []

    for column in table.columns:
        for shift in range(1, context_size + 1):
            s = table[column].shift(shift)
            s.name = f"{column}-{shift}"

            series.append(s)
            output_names.append(s.name)

    table = pandas.concat([table, *series], axis=1)
    table.dropna(inplace=True)

    return table[output_names], table[original]


train_val_input, train_val_output = extract_xy(train_val, context_size=CONTEXT_SIZE)
train_val_input.shape, train_val_output.shape

((1246, 20), (1246, 5))

In [4]:
train_val_input

,open-1,open-2,open-3,open-4,high-1,high-2,high-3,high-4,low-1,low-2,low-3,low-4,close-1,close-2,close-3,close-4,volume-1,volume-2,volume-3,volume-4
4,54.58,52.87,54.58,54.33,55.08,53.95,54.89,54.83,53.95,51.99,53.00,54.33,54.45,53.32,53.44,54.77,255840.0,336550.0,404480.0,417520.0
5,54.45,54.58,52.87,54.58,55.97,55.08,53.95,54.89,54.45,53.95,51.99,53.00,55.71,54.45,53.32,53.44,600270.0,255840.0,336550.0,404480.0
6,56.60,54.45,54.58,52.87,57.04,55.97,55.08,53.95,56.16,54.45,53.95,51.99,56.72,55.71,54.45,53.32,515500.0,600270.0,255840.0,336550.0
7,57.35,56.60,54.45,54.58,58.17,57.04,55.97,55.08,57.10,56.16,54.45,53.95,57.61,56.72,55.71,54.45,685580.0,515500.0,600270.0,255840.0
8,57.67,57.35,56.60,54.45,57.86,58.17,57.04,55.97,56.85,57.10,56.16,54.45,57.42,57.61,56.72,55.71,338400.0,685580.0,515500.0,600270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,68.16,68.25,68.70,68.25,68.43,68.34,68.70,68.61,67.61,67.70,68.07,67.52,67.98,67.98,68.43,68.25,617722.0,728928.0,574287.0,887468.0
1246,67.98,68.16,68.25,68.70,69.80,68.43,68.34,68.70,67.98,67.61,67.70,68.07,69.61,67.98,67.98,68.43,1057401.0,617722.0,728928.0,574287.0
1247,69.80,67.98,68.16,68.25,69.80,69.80,68.43,68.34,68.89,67.98,67.61,67.70,69.52,69.61,67.98,67.98,756305.0,1057401.0,617722.0,728928.0
1248,69.52,69.80,67.98,68.16,69.80,69.80,69.80,68.43,69.25,68.89,67.98,67.61,69.43,69.52,69.61,67.98,707087.0,756305.0,1057401.0,617722.0


In [5]:
train_val_output

,open,high,low,close,volume
4,54.45,55.97,54.45,55.71,600270
5,56.60,57.04,56.16,56.72,515500
6,57.35,58.17,57.10,57.61,685580
7,57.67,57.86,56.85,57.42,338400
8,57.29,57.29,56.16,56.47,370320
...,...,...,...,...,...
1245,67.98,69.80,67.98,69.61,1057401
1246,69.80,69.80,68.89,69.52,756305
1247,69.52,69.80,69.25,69.43,707087
1248,69.52,69.52,68.80,69.16,767974


In [6]:
from sklearn.model_selection import train_test_split


train_input, val_input, train_output, val_output = train_test_split(
    train_val_input,
    train_val_output,
    train_size=0.8,
    random_state=42,
    shuffle=True,
)
train_input.shape, val_input.shape, train_output.shape, val_output.shape

((996, 20), (250, 20), (996, 5), (250, 5))

In [7]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
train_input_scaled = scaler.fit_transform(train_input)
val_input_scaled = scaler.transform(val_input)

In [8]:
import numpy


rng = numpy.random.default_rng(42)
hidden_size = 32
w = rng.random((train_val_input.shape[1], hidden_size))
b = rng.random((1, hidden_size))
w.shape, b.shape

((20, 32), (1, 32))

In [9]:
def sigmoid(x: numpy.ndarray) -> numpy.ndarray:
    return 1 / (1 + numpy.exp(-x))


def transform_input(x: numpy.ndarray, *, w: numpy.ndarray, b: numpy.ndarray) -> numpy.ndarray:
    h = sigmoid(x @ w + b)
    return numpy.concat([x, h], axis=1)


z = transform_input(train_input_scaled, w=w, b=b)

In [10]:
from typing import TypedDict

from sklearn.linear_model import RidgeCV
from sklearn.metrics import (
    mean_absolute_percentage_error,
    root_mean_squared_error,
    r2_score,
)


class Metrics(TypedDict):
    rmse: float
    mape: float
    r2: float


def metrics(true: pandas.DataFrame, pred: numpy.ndarray) -> Metrics:
    return {
        "rmse": root_mean_squared_error(true, pred),
        "mape": mean_absolute_percentage_error(true, pred),
        "r2": r2_score(true, pred),
    }


model = RidgeCV()
model.fit(z, train_output)
metrics(train_output, model.predict(z))

{'rmse': 68540.82813304468,
 'mape': 0.09578651495882205,
 'r2': 0.9069412189411332}

In [11]:
metrics(val_output, model.predict(transform_input(val_input_scaled, w=w, b=b)))

{'rmse': 81447.20701430114,
 'mape': 0.09404641516900833,
 'r2': 0.887994233115783}

In [12]:
test = pandas.read_csv("../data/vn30/GAS_test.csv")
test.drop("time", axis=1, inplace=True)
test_input, test_output = extract_xy(test, context_size=CONTEXT_SIZE)

test_input_scaled = scaler.transform(test_input)
metrics(test_output, model.predict(transform_input(test_input_scaled, w=w, b=b)))

{'rmse': 105654.44318901752,
 'mape': 0.09681195484999211,
 'r2': 0.8345349060574592}